# The Agents of FloAI

This notebook shows different kinds of agents that can be built using flo-ai. These agents have their own specialities, and this notebook tries to explain them.

Here are the major types of agents:

1. Agentic Agents (`kind: agentic`): Any agent created in flo-ai by default tries to be an agentic agent. This agent always needs a tool, meaning an agent without a tool if specified as agentic agent whill throw an exception

2. LLM Agents (`kind: llm`): These agents are simply an LLM which can answer any questions asked to it. The agents dont except tools. If tool is passed to an agent of type llm, they are ignored.

3. Tool LLM (`kind: tool`): These agents are just tools or functions that can be executed on the current state. Within the tool will be given the current state as the input, meaning the history of what happened in the flo until now

In [8]:
import os , sys
print("Current working directory:", os.getcwd())
print("Python path:", sys.path)
try:
    from flo_ai.common.flo_logger import get_logger
    print("Successfully imported get_logger")
except ImportError as e:
    print("Failed to import get_logger:", str(e))
    # You can add more debugging information here

Current working directory: /Users/thomas/Documents/projects/flo-4/flo-ai/examples
Python path: ['/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/thomas/Documents/projects/flo-4/flo-ai/flo4/lib/python3.11/site-packages', '/Users/thomas/Documents/projects/flo-4/flo-ai']
Successfully imported get_logger


In [1]:
from flo_ai import Flo
from flo_ai import FloSession
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from flo_ai.common.flo_logger import FloLogger
logger1=FloLogger.get_logger(__name__)
logger1.error("This is an error message.")

2024-09-12 13:58:30,392 - __main__ - ERROR - This is an error message.


In [16]:
import logging
# logging.warning("Remain calm!")
logging.basicConfig(
    filename="app.log",
    encoding="utf-8",
    filemode="a",
    format="{asctime} - {levelname} - {message}",
    style="{",
    datefmt="%Y-%m-%d %H:%M",
    # level=logging.WARNING
)
logging.warning("Something went wrong!")

## Setup
Create the Flo session, setup tools

In [2]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
session = FloSession(llm)

session.register_tool(
    name="InternetSearchTool", 
    tool=TavilySearchResults()
)

## Agentic Agent (agentic)

Here we are gonna create a simple weather assitant flo agent that can check the whether by an internet searching tool.

As you can see the kind is `agentic`

In [3]:
simple_weather_checking_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: weather-assistant
agent:
    name: Weather Assistant
    kind: agentic
    job: >
      Given the city name you are capable of answering the latest whether this time of the year by searching the internet
    tools:
      - name: InternetSearchTool
"""

In [ ]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
flo = Flo.build(session, simple_weather_checking_agent)


class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"My custom handler, token: {token}")


handler = MyCustomHandler()
config = {
    'callbacks' : [handler]
}

query="Whats the whether in california"

flo.invoke(query=query, config=config)

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        """Run on agent action."""
        
# prompt = ChatPromptTemplate.from_messages(["Tell me a joke about {animal}"])

# To enable streaming, we pass in `streaming=True` to the ChatModel constructor
# Additionally, we pass in our custom handler as a list to the callbacks parameter
# model = ChatOpenAI(streaming=True, callbacks=[MyCustomHandler()])

# chain = prompt | model

# response = chain.invoke({"animal": "bears"})

In [4]:
# from langchain_core.callbacks import StdOutCallbackHandler
# handler = StdOutCallbackHandler()

from flo_ai.common.flo_callback_handler import FloCallbackHandler

flo = Flo.build(session, simple_weather_checking_agent)

handler = FloCallbackHandler(log_level="INFO")

config = {
    'callbacks' : [handler]
}

query="Whats the whether in california"

flo.invoke(query=query, config=config)

2024-09-13 01:15:31,867 - FloCallback - INFO - Chain started. Inputs: {'messages': [HumanMessage(content='Whats the whether in california')]}
2024-09-13 01:15:31,900 - FloCallback - INFO - Chain started. Inputs: {'input': ''}
2024-09-13 01:15:31,913 - FloCallback - INFO - Chain started. Inputs: {'input': ''}
2024-09-13 01:15:31,921 - FloCallback - INFO - Chain started. Inputs: {'input': ''}
2024-09-13 01:15:31,926 - FloCallback - INFO - Chain started. Inputs: {'input': ''}
2024-09-13 01:15:31,930 - FloCallback - INFO - Chain ended. Outputs: []
2024-09-13 01:15:31,932 - FloCallback - INFO - Chain ended. Outputs: {'agent_scratchpad': []}
2024-09-13 01:15:31,935 - FloCallback - INFO - Chain ended. Outputs: {'messages': [HumanMessage(content='Whats the whether in california')], 'intermediate_steps': [], 'agent_scratchpad': []}
2024-09-13 01:15:31,937 - FloCallback - INFO - Chain started. Inputs: {'messages': [HumanMessage(content='Whats the whether in california')], 'intermediate_steps': [



> Entering new AgentExecutor chain...


2024-09-13 01:15:33,913 - FloCallback - INFO - LLM ended. Output: [[ChatGenerationChunk(generation_info={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857'}, message=AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_uQzFA7BzlZZef6l75Zn6E2og', 'function': {'arguments': '{"query":"California weather October 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857'}, id='run-25eea023-57c1-4693-a62b-d99c2208555d', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'California weather October 2023'}, 'id': 'call_uQzFA7BzlZZef6l75Zn6E2og', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'tavily_search_results_json', 'args': '{"query":"California weather October 2023"}', 'id': 'call_uQzFA7BzlZZef6l75Zn6E2og', 'index': 0, 'type': 'tool_call_ch


Invoking: `tavily_search_results_json` with `{'query': 'California weather October 2023'}`




2024-09-13 01:15:38,441 - FloCallback - INFO - Tool ended. Output: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726170321, 'localtime': '2024-09-12 12:45'}, 'current': {'last_updated_epoch': 1726170300, 'last_updated': '2024-09-12 12:45', 'temp_c': 29.1, 'temp_f': 84.4, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1008.0, 'pressure_in': 29.77, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 11, 'cloud': 0, 'feelslike_c': 27.0, 'feelslike_f': 80.5, 'windchill_c': 26.0, 'windchill_f': 78.7, 'heatindex_c': 24.7, 'heatindex_f': 76.4, 'dewpoint_c': 1.0, 'dewpoint_f': 33.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 7.0, 'gust_mph': 6.7, 'gust_kph': 1

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726170321, 'localtime': '2024-09-12 12:45'}, 'current': {'last_updated_epoch': 1726170300, 'last_updated': '2024-09-12 12:45', 'temp_c': 29.1, 'temp_f': 84.4, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1008.0, 'pressure_in': 29.77, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 11, 'cloud': 0, 'feelslike_c': 27.0, 'feelslike_f': 80.5, 'windchill_c': 26.0, 'windchill_f': 78.7, 'heatindex_c': 24.7, 'heatindex_f': 76.4, 'dewpoint_c': 1.0, 'dewpoint_f': 33.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 7.0, 'gust_mph': 6.7, 'gust_kph': 10.8}}"}, {'url': 'https://www.timeanddate.com/weather/usa/los-angel

2024-09-13 01:15:40,497 - FloCallback - INFO - LLM ended. Output: [[ChatGenerationChunk(text='The current weather in California is sunny with a temperature of approximately 29.1°C (84.4°F). The humidity is low at 11%, and there is no precipitation. Winds are light, coming from the north at about 2.2 mph.\n\nFor a broader overview of October 2023, temperatures in California have varied, with highs reaching up to 92°F on October 5. The weather has generally been warm, typical for this time of year.\n\nFor more detailed forecasts and historical data, you can check the following sources:\n- [Weather API](https://www.weatherapi.com/)\n- [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023)\n- [Meteoprog](https://www.meteoprog.com/weather/California-california/month/october/)\n- [World Weather](https://world-weather.info/forecast/usa/california/october-2023/)', generation_info={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system

The current weather in California is sunny with a temperature of approximately 29.1°C (84.4°F). The humidity is low at 11%, and there is no precipitation. Winds are light, coming from the north at about 2.2 mph.

For a broader overview of October 2023, temperatures in California have varied, with highs reaching up to 92°F on October 5. The weather has generally been warm, typical for this time of year.

For more detailed forecasts and historical data, you can check the following sources:
- [Weather API](https://www.weatherapi.com/)
- [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023)
- [Meteoprog](https://www.meteoprog.com/weather/California-california/month/october/)
- [World Weather](https://world-weather.info/forecast/usa/california/october-2023/)

> Finished chain.


{'messages': [HumanMessage(content='Whats the whether in california')],
 'output': 'The current weather in California is sunny with a temperature of approximately 29.1°C (84.4°F). The humidity is low at 11%, and there is no precipitation. Winds are light, coming from the north at about 2.2 mph.\n\nFor a broader overview of October 2023, temperatures in California have varied, with highs reaching up to 92°F on October 5. The weather has generally been warm, typical for this time of year.\n\nFor more detailed forecasts and historical data, you can check the following sources:\n- [Weather API](https://www.weatherapi.com/)\n- [Time and Date](https://www.timeanddate.com/weather/usa/los-angeles/historic?month=10&year=2023)\n- [Meteoprog](https://www.meteoprog.com/weather/California-california/month/october/)\n- [World Weather](https://world-weather.info/forecast/usa/california/october-2023/)'}

## LLM Agent (agentic)

Here we are gonna create a simple llm math assitant flo agent that can check answer any math question

As you can see the kind is `llm`

In [15]:
simple_llm_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: Ask llm anything
    kind: llm
    job: >
      You are a high school maths teacher. Answer any questions the students ask 
"""

In [17]:
flo = Flo.build(session, simple_llm_agent)

flo.invoke("What is pythagorus theorum")
# print(flo.invoke("What is pythagorus theorum"))

2024-09-12 23:10:45,889 - Flo.build - INFO - Building Flo instance from YAML
2024-09-12 23:10:45,889 - Flo.build - INFO - Building Flo instance from YAML
2024-09-12 23:10:45,905 - Flo - INFO - Invoking query: What is pythagorus theorum


"Pythagoras' Theorem is a fundamental principle in geometry that relates to right-angled triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. \n\nThe theorem can be expressed with the formula:\n\n\\[ c^2 = a^2 + b^2 \\]\n\nwhere:\n- \\( c \\) is the length of the hypotenuse,\n- \\( a \\) and \\( b \\) are the lengths of the other two sides.\n\nThis theorem is useful for calculating the length of one side of a right triangle if the lengths of the other two sides are known. Would you like to see an example of how to use it?"

#

In [7]:
from typing import Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

class PrintStateTool(BaseTool):
    name = "printStateTool"
    description = "Just print the state"

    def _run(
        self, **kwargs
    ) -> str:
        print(kwargs)
        return "Tool call success"
    
session.register_tool(
    name="printStateTool", 
    tool=PrintStateTool()
)
    
simple_tool_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: tool-to-print-state
    kind: tool
    tools:
        - name: printStateTool
"""

flo = Flo.build(session, simple_tool_agent)

print(flo.invoke("Print what I am saying"))

2024-09-12 15:51:43,357 - Flo.build - INFO - Building Flo instance from YAML
2024-09-12 15:51:43,357 - Flo.build - INFO - Building Flo instance from YAML
2024-09-12 15:51:43,361 - Flo - INFO - Invoking query: Print what I am saying


{'messages': [HumanMessage(content='Print what I am saying')]}
Tool call success
